# Energy system optimisation with oemof - how to collect and store results

### Import necessary modules

In [ ]:
import os
import pandas as pd
from oemof.solph import (Sink, Source, Transformer, Bus, Flow, Model,
                         EnergySystem)

import pickle

### Specify solver

In [ ]:
solver = 'cbc'

### Create an energy system and optimize the dispatch at least costs.

In [ ]:
# initialize and provide data
datetimeindex = pd.date_range('1/1/2016', periods=24*10, freq='H')
energysystem = EnergySystem(timeindex=datetimeindex)
filename = 'input_data.csv'
data = pd.read_csv(filename, sep=",")

### Create and add components to energysystem

In [ ]:
# resource buses
bcoal = Bus(label='coal', balanced=False)
bgas = Bus(label='gas', balanced=False)
boil = Bus(label='oil', balanced=False)
blig = Bus(label='lignite', balanced=False)

# electricity and heat
bel = Bus(label='bel')
bth = Bus(label='bth')

energysystem.add(bcoal, bgas, boil, blig, bel, bth)

In [ ]:
# an excess and a shortage variable can help to avoid infeasible problems
energysystem.add(Sink(label='excess_el', inputs={bel: Flow()}))
# shortage_el = Source(label='shortage_el',
#                      outputs={bel: Flow(variable_costs=200)})

# sources
energysystem.add(Source(label='wind', outputs={bel: Flow(
    fix=data['wind'], nominal_value=66.3)}))

energysystem.add(Source(label='pv', outputs={bel: Flow(
    fix=data['pv'], nominal_value=65.3)}))

# demands (electricity/heat)
energysystem.add(Sink(label='demand_el', inputs={bel: Flow(
    nominal_value=85, fix=data['demand_el'])}))

energysystem.add(Sink(label='demand_th',
                 inputs={bth: Flow(nominal_value=40,
                                   fix=data['demand_th'],
                                   fixed=True)}))

In [ ]:
# power plants
energysystem.add(Transformer(
    label='pp_coal',
    inputs={bcoal: Flow()},
    outputs={bel: Flow(nominal_value=20.2, variable_costs=25)},
    conversion_factors={bel: 0.39}))

energysystem.add(Transformer(
    label='pp_lig',
    inputs={blig: Flow()},
    outputs={bel: Flow(nominal_value=11.8, variable_costs=19)},
    conversion_factors={bel: 0.41}))

energysystem.add(Transformer(
    label='pp_gas',
    inputs={bgas: Flow()},
    outputs={bel: Flow(nominal_value=41, variable_costs=40)},
    conversion_factors={bel: 0.50}))

energysystem.add(Transformer(
    label='pp_oil',
    inputs={boil: Flow()},
    outputs={bel: Flow(nominal_value=5, variable_costs=50)},
    conversion_factors={bel: 0.28}))

In [ ]:
# combined heat and power plant (chp)
energysystem.add(Transformer(
    label='pp_chp',
    inputs={bgas: Flow()},
    outputs={bel: Flow(nominal_value=30, variable_costs=42),
             bth: Flow(nominal_value=40)},
    conversion_factors={bel: 0.3, bth: 0.4}))

In [ ]:
# heat pump with a coefficient of performance (COP) of 3
b_heat_source = Bus(label='b_heat_source')
energysystem.add(b_heat_source)

energysystem.add(Source(label='heat_source', outputs={b_heat_source: Flow()}))

cop = 3
energysystem.add(Transformer(
    label='heat_pump',
    inputs={bel: Flow(),
            b_heat_source: Flow()},
    outputs={bth: Flow(nominal_value=10)},
    conversion_factors={bel: 1/3, b_heat_source: (cop-1)/cop}))

### Optimization

In [ ]:
# create optimization model based on energy_system
optimization_model = Model(energysystem=energysystem)

# solve problem
optimization_model.solve(solver=solver,
                         solve_kwargs={'tee': True, 'keepfiles': False})

### Write results into energysystem.results object for later

In [ ]:
energysystem.results['main'] = solph.processing.results(optimization_model)
energysystem.results['meta'] = solph.processing.meta_results(optimization_model)

In [ ]:
string_results = solph.views.convert_keys_to_strings(energysystem.results['main'])

### Save results - Dump the energysystem (to ~/home/user/.oemof by default)
Specify path and filename if you do not want to overwrite

In [ ]:
energysystem.dump(dpath=None, filename=None)